In [1]:
# importing the necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [8]:
newsrooms = pd.read_csv('../data/raw/newsrooms.csv')
newsrooms.columns = ['site', 'monthly_visits', 'country', 'country_of_pub']
newsrooms.dropna(inplace=True)
newsrooms = newsrooms.drop_duplicates(subset = 'site')
newsrooms = newsrooms.sort_values(by=['country_of_pub', 'monthly_visits'], ascending=False).reset_index().drop('index', axis=1)

In [9]:
# set dfs with sites
UK = newsrooms[newsrooms['country_of_pub']=='UK'].reset_index().drop('index', axis=1)
USA = newsrooms[newsrooms['country_of_pub']=='USA'].reset_index().drop('index', axis=1)
India = newsrooms[newsrooms['country_of_pub']=='India'].reset_index().drop('index', axis=1)
SA = newsrooms[newsrooms['country_of_pub']=='South Africa'].reset_index().drop('index', axis=1)

In [10]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np
import datetime

In [11]:
def send_request(url):
    '''Send request and sleep if a connection error comes up.'''
    # send request
    try:
        r1 = requests.get(url)
    except (ConnectionError, OSError, TimeoutError) as e:
        print("Encountered an error, F*** YOU GOOGLE, sleeping . . .")
        sleep(90)
        try:
            r1 = requests.get(url)
        except (ConnectionError, OSError, TimeoutError) as e:
            print("Encountered an error, F*** YOU GOOGLE, sleeping . . .")
            sleep(90)
            try:
                r1 = requests.get(url)
            except (ConnectionError, OSError, TimeoutError) as e:
                print("Encountered an error, F*** YOU GOOGLE, sleeping . . .")
                sleep(90)
                r1 = requests.get(url)
    # get html content
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html5lib')
    
    return soup1

In [46]:
year = 2020
month_ranges = [[[1, 1, year], [31, 1, year]], [[1, 2, year], [28, 2, year]],
                [[1, 3, year], [31, 3, year]],
                [[1, 4, year], [30, 4, year]],
                [[1, 5, year], [31, 5, year]],
                [[1, 6, year], [30, 6, year]],
                [[1, 7, year], [31, 7, year]],
                [[1, 8, year], [31, 8, year]],
                [[1, 9, year], [30, 9, year]],
                [[1, 10, year], [31, 10, year]],
                [[1, 11, year], [30, 11, year]],
                [[1, 12, year], [31, 12, year]]]

In [45]:
# loop
%time
from time import sleep
from random import randint
# Country: 
query = ['women+%7C+woman+%7C+girl+%7C+girls+%7C+female+%7C+females+%7C+lady+%7C+ladies+%7C+she+%7C+her+']
# sites = list(USA.site.unique().flatten())[10:12]
sites = list(SA.site.unique().flatten())
starting_result = range(0, 10, 10)
# starting_result = [0, 10]


for site in sites:
    
    urls = []
    headlines = []
    subtitles = []
    times = []
    scrape_dates = []
    websites = []

    for i in range(len(month_ranges[0:3])):
        #start_date = month_ranges[i][0]
        start_day = month_ranges[i][0][0]
        start_month = month_ranges[i][0][1]
        start_year = month_ranges[i][0][2]

        end_day = month_ranges[i][1][0]
        end_month = month_ranges[i][1][1]
        end_year = month_ranges[i][1][2]
        #end_date = month_ranges[i][0]
    
        for start in starting_result:
            # keep track
            print("Scraping page", str((start+10)/10), "of", site, "google news results.")
            # change url
            url = f'https://www.google.com/search?q={query[0]}+site%3A{site}&biw=1188&bih=962&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{start_month}%2F{start_day}%2F{start_year}%2Ccd_max%3A{end_month}%2F{end_day}%2F{end_year}&tbm=nws'
        # f'https://www.google.com/search?q={query[0]}+site:{site}&tbm=nws&sxsrf=ALeKk02yZ8PpgaqZrEk_lf2SjYFoIO2UFA:1605438024548&ei=SAqxX9v1IIv0kwXI44gY&start={start}&sa=N&ved=0ahUKEwjbuaXZsoTtAhUL-qQKHcgxAgM4FBDy0wMIhgE&biw=1188&bih=962&dpr=1.25'
           # print(url)
            # sleep
            sleep(randint(10,60))
    #         # send request
    #         try:
    #             r1 = requests.get(url)
    #         except ConnectionError:
    #             sleep(90)
    #             r1 = requests.get(url)
    #         # get html content
    #         coverpage = r1.content
    #         soup1 = BeautifulSoup(coverpage, 'html5lib')
            soup1 = send_request(url)
            # check whether google is trying to redirect to captcha
            if "Our systems have detected unusual traffic from your computer network" in str(soup1):
                sleep(90)
                soup1 = send_request(url)
            # continue
            coverpage_news = soup1.find_all('div', class_='ZINbbc xpd O9g5cc uUPGi')
            if len(coverpage_news)>0:
                # get info for each of 10 google results
                for i in range(len(coverpage_news)):
                    # url
                    url = coverpage_news[i].a['href'][7:]
                    urls.append(url)
                    # headline
                    headline = coverpage_news[i].a.text
                    headlines.append(headline)
                    # subtitle
                    subtitle = coverpage_news[i].text.split('�')[1].strip().strip('"').strip('.')
                    subtitles.append(subtitle)
                    # time
                    time = coverpage_news[i].span.text
                    times.append(time)
                    # scrape date
                    scrape_date = datetime.datetime.now().strftime("%d/%m/%Y")
                    scrape_dates.append(scrape_date)
                    # website
                    websites.append(site)
                # keep track
                print(str(start + 10) + " results of site " + str(site) + " scraped successfully!")
            else:
                break
                
        # dictionary of results
        headlines_dict = {"url": urls, "headline": headlines,  "subtitle": subtitles, "time": times, "scrape_date": scrape_dates, "site": websites}
        # df of results
        headlines_df = pd.DataFrame.from_dict(headlines_dict)
        number_of_articles = len(headlines_df)
        # save results to csv
        headlines_df.to_csv(f'../data/{number_of_articles}_{site}_articles.csv')
        # print final statement
        print("Saved data from " + str(number_of_articles) + " articles of "  + str(site) + " to .csv!")
        
    print("Scrape complete!")

//www.google.com/search?q=women+%7C+woman+%7C+girl+%7C+girls+%7C+female+%7C+females+%7C+lady+%7C+ladies+%7C+she+%7C+her++site%3AEwn.co.za&biw=1188&bih=962&source=lnt&tbs=cdr%3A1%2Ccd_min%3A1%2F1%2F2020%2Ccd_max%3A1%2F31%2F2020&tbm=nws
Scraping page 1.0 of Ewn.co.za google news results.
https://www.google.com/search?q=women+%7C+woman+%7C+girl+%7C+girls+%7C+female+%7C+females+%7C+lady+%7C+ladies+%7C+she+%7C+her++site%3AEwn.co.za&biw=1188&bih=962&source=lnt&tbs=cdr%3A1%2Ccd_min%3A2%2F1%2F2020%2Ccd_max%3A2%2F28%2F2020&tbm=nws
Scraping page 1.0 of Ewn.co.za google news results.
https://www.google.com/search?q=women+%7C+woman+%7C+girl+%7C+girls+%7C+female+%7C+females+%7C+lady+%7C+ladies+%7C+she+%7C+her++site%3AEwn.co.za&biw=1188&bih=962&source=lnt&tbs=cdr%3A1%2Ccd_min%3A3%2F1%2F2020%2Ccd_max%3A3%2F31%2F2020&tbm=nws
Scraping page 1.0 of Thesouthafrican.com google news results.
https://www.google.com/search?q=women+%7C+woman+%7C+girl+%7C+girls+%7C+female+%7C+females+%7C+lady+%7C+ladies+%7C+she

In [24]:
headlines_dict = {"url": urls, "headline": headlines,  "subtitle": subtitles, "time": times, "scrape_date": scrape_dates, "site": websites}
# df of results
headlines_df = pd.DataFrame.from_dict(headlines_dict)
number_of_articles = len(headlines_df)
# save results to csv
headlines_df.to_csv(f'../data/{number_of_articles}_{site}_articles.csv')

In [20]:
list(SA.site.unique().flatten())[12:]

['Dailymaverick.co.za',
 'Vodacom.co.za',
 'Sowetanlive.co.za',
 'mg.co.za',
 'MyBroadband.co.za',
 'Moneyweb.co.za',
 'Businesslive.co.za',
 'Enca.com',
 'Capetownetc.com',
 'iol.co.za',
 'heraldlive.co.za',
 'dispatchlive.co.za',
 'dailysun.co.za',
 'cbn.co.za',
 'lowvelder.co.za',
 'zululandobserver.co.za',
 'ziwaphi.com']

[]

In [334]:
coverpage_news[i].a['href'][7:]

'https://www.theblaze.com/news/leftist-woman-spits-in-face-of-cop&sa=U&ved=2ahUKEwj_2pujuo7tAhUMI6wKHaZdAtUQxfQBMAl6BAgCEAE&usg=AOvVaw2TSZgMP4nTF5XNFsvmKzCh'

In [180]:
coverpage_news = soup1.find_all('div', class_='ZINbbc xpd O9g5cc uUPGi')
# coverpage_news = soup1.find_all('div', class_='kCrYT')


In [352]:
site = random.choice(sites)

In [355]:
headlines_dict = {"url": [url], "headline": [headline],  "subtitle": [subtitle], "time": [time], "scrape_date": [scrape_date], "site":[site]}
headlines_df = pd.DataFrame.from_dict(headlines_dict)
number_of_articles = len(headlines_df)
headlines_df.to_csv(f'../data/{number_of_articles}_articles.csv')

In [372]:
print(headline)
print(url)
print(time)
print(subtitle)
print(scrape_date)

19-year-old woman ends life in UP''s Bulandshahr, blames three men in suicide noteOutlook India
https://www.outlookindia.com/newsscroll/19yearold-woman-ends-life-in-ups-bulandshahr-blames-three-men-in-suicide-note/1977017&sa=U&ved=2ahUKEwiM16CdxY7tAhVDXawKHd_cBnU4ChDF9AEwB3oECAAQAQ&usg=AOvVaw1n2cj0aRpqzPcYPDnrIMth
2 giorni fa
Bulandshahr (UP), Nov 16 (PTI) A 19-year-old woman allegedly died by suicide in a ... She left behind a suicide note blaming three people for her death. ... and statement) of the CrPC, the girl completely denied the incident
19/11/2020


In [371]:
# print(soup1.prettify())
i = 7
url = coverpage_news[i].a['href'][7:]
headline = coverpage_news[i].a.text
time = coverpage_news[i].span.text
subtitle = coverpage_news[i].text.split('�')[1].strip().strip('"').strip('.')
scrape_date = datetime.datetime.now().strftime("%d/%m/%Y")